This notebook is made just for learning purposes. The original notebook is in [here](https://www.kaggle.com/code/mobinapoulaei/autogloun-t8-dslanders). It works just 1% of the whole competetion data.

In [1]:
!pip install ray==2.10.0
!pip install autogluon.tabular
!pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 25.6 MB/s eta 0:00:00
  Attempting uninstall: ray
    Found existing installation: ray 2.24.0
    Uninstalling ray-2.24.0:
      Successfully uninstalled ray-2.24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.1/312.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.8/234.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 81.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-

In [2]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

In [3]:
train_df = pd.read_csv('/kaggle/input/playground-series-s4e8/train.csv').drop('id', axis=1).sample(frac = 0.001)
test_df = pd.read_csv('/kaggle/input/playground-series-s4e8/test.csv').drop('id', axis=1).sample(frac = 0.001)

In [4]:
predictor = TabularPredictor(label='class', #label column
                            eval_metric='mcc', #evaluation metric:  mcc
                            problem_type='binary') #problem type: binary classification

predictor.fit(train_df,
              presets='best_quality',
              time_limit=60,
              verbosity=0,
              excluded_model_types=['KNN'],
             # ag_args_fit={'num_gpus': 1
              )

results = predictor.fit_summary()

No path specified. Models will be saved in: "AutogluonModels/ag-20241121_092120"
2024-11-21 09:21:23,711	INFO worker.py:1752 -- Started a local Ray instance.
(_ray_fit pid=353) 	Ran out of time, early stopping on iteration 197. Best iteration is:
(_ray_fit pid=353) 	[141]	valid_set's binary_logloss: 0.144011	valid_set's mcc: 0.953318
(_ray_fit pid=471) 	Ran out of time, early stopping on iteration 224. Best iteration is: [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=471) 	[168]	valid_set's binary_logloss: 0.125094	valid_set's mcc: 0.947516 [repeated 4x across cluster]


*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0    LightGBMXT_BAG_L1   0.976685         mcc       0.516947  23.332334                0.516947          23.332334            1       True          1
1  WeightedEnsemble_L2   0.976685         mcc       0.520202  23.338614                0.003255           0.006280            2       True          2
2  WeightedEnsemble_L3   0.976685         mcc       0.521582  23.496604                0.004635           0.164270            3       True          4
3    LightGBMXT_BAG_L2   0.968891         mcc       0.786747  33.482883                0.269801          10.150550            2       True          3
Number of models trained: 4
Types of models trained:
{'WeightedEnsembleModel', 'StackerEnsembleModel_LGB'}
Bagging used: True  (with 8 folds)
Multi-layer stack-ensembling used: True  (with

In [5]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L1,0.976685,mcc,0.516947,23.332334,0.516947,23.332334,1,True,1
1,WeightedEnsemble_L2,0.976685,mcc,0.520202,23.338614,0.003255,0.006280,2,True,2
2,WeightedEnsemble_L3,0.976685,mcc,0.521582,23.496604,0.004635,0.164270,3,True,4
3,LightGBMXT_BAG_L2,0.968891,mcc,0.786747,33.482883,0.269801,10.150550,2,True,3


In [6]:
y_pred = predictor.predict(test_df)

In [7]:
sub = pd.read_csv('/kaggle/input/playground-series-s4e8/sample_submission.csv')
sub['class'] = y_pred
sub.to_csv('submission.csv', index=False)